# Zawaansowane widoki

## Agenda
1. Request & response
2. Do czego służą widoki ?
3. Rodzaje widoków
4. Django template language
5. Tworzenie widoków w naszym projekcie
6. Widoki generyczne

### Request vs. response
Jest to podstawowy sposób komunikacji odbywającej się w protokole HTTP. Gdy chcemy uzyskać dostęp do jakiegoś miejsca np. `http://www.google.com` wysyłamy **żądanie** (**reqest**) do serwera, który hostuje ten adres. Ten serwer odbiera nasz **request**, przetwarza go i zwraca odpowiedni **response**.

W Django mamy wbudowane obiekty, które pozwalają nam obsługiwać zarówno żądania jak i odpowiedzi na te żądania.

**HttpRequest**
W Django za obsługę żądania HTTP odpowiada obiekt HttpRequest. Klasa definiująca ten obiekt znajduje się w module django.http.
Zawiera wszystkie pola nagłówków żądania. Te pola możesz wykorzystywać w swojej aplikacji.
Obiekt HttpRequest jest obsługiwany przez framework. W większości przypadków nie będziecie o nim nawet myśleć ;-)

**HttpResponse**
Za obsługę odpowiedzi w Django odpowiada obiekt HttpResponse. Klasa definiująca ten obiekt znajduje się w module django.http.

### Do czego służą widoki ?
Widok jest to kluczowy komponent frameworku, pisałem już o nich w pierwszym rozdziale. Są to funkcję / klasy które przyjmują obiekt **HttpRequest** przetwarzają go i zwracają obiekt **HttpResponse**. Każdy widok musi być przypisany do URL'a. Widoki są używane do rzeczy takich jak wyciąganie rzeczy z bazy, modyfikowanie danych w bazie, obsługa formularzy, zwracanie HTML'a ..

Wróćmy zatem do naszego projektu. Stworzyliśmy ostanio model `Article` oraz nauczyliśmy się jak uzyskiwać infromacje z tego modelu używając `querysetów`, wykorzystajmy tą wiedzę tworząc widok, dostępny pod adresem `/articles`, który będzie zwracał listę wszystkich artykułów.
W pliku `views.py` napiszmy:

```python
from django.http import HttpResponse
from django.shortcuts import render
from my_app.models import Article

...

def articles(request):
    articles = Article.objects.all()
    return render(request, "articles.html", {
        "articles": articles
    })
```
Stworzyliśmy widok **articles**, który przyjmuje obowiązkowy parametr **request**. W widoku wyciągamy z bazy wszystkie artykuły z modelu **Article**. Zwracamy response używając funkcji **render()** zaimportowanej z `django.shortcuts`, ta funkcja przyjmuje 3 parametry:
* obiekt **request**
* plik html w który ma być wyrenderowany. Domyślna lokalizacja w której Django szuka templatek to jest folder `templates` wewnątrz aplikacji. Dlatego plik **articles.htm** znajduje się w **my_app/templates/articles.html**
* zmienne, które zostaną przekazane do tego html'a. Zbiór tych zmiennych nazywamy **kontekstem** nazwy które piszemy w cudzysłowie to będą nazwy do których będziemy się odnosili w HTML'u

Stwórzmy więc plik **articles.html** w folderze **templates** wewnątrz aplikacji **my_app**.

```html
<!DOCTYPE html>
<html lang="en">
<head>
    <title>Lista artykułów</title>
</head>
<body>
    <h1>Lista artykułów</h1>
    <hr>
    {% for article in articles %}
        <h5>{{ article.title }}</h5>
        <p>Author: <span>{{ article.author.username }}</span></p>
        <p> 
            Likes: <span style="font-weight: bold;">{{ article.likes }}</span> 
            Dislikes: <span style="font-weight: bold;">{{ article.dislikes }}</span>
        </p>
        <br>
        <br>
    {% endfor %}
</body>
</html>
```

Trochę dziwny ten HTML, prawda ;) ? Django posiada swój silnik zwany **Django Template Language** który pozwala pisać zwykły html z dodatkami takimi jak wyświetlanie zmiennych przekazanych w kontekscie, używanie różnych **tagów** takich jak pętla czy instrukcja warunkowa oraz wiele więcej.


Musimy przypisać nasz widok pod wybrany przez nas URL. Przejdźmy do pliku **urls.py** znajdującego się w folderze **new_project** i napisz:

```python
from django.contrib import admin
from django.urls import path

from my_app import views 

urlpatterns = [
    path('admin/', admin.site.urls),
    path("", views.home_view, name="home"),
    path("articles/", views.articles, name="articles"), # <-- ta linia została dodana
]
```


Uruchom serwer i przejdź na adres http://localhost:8000/articles
Powinien się wyświetlić następujący rezultat:
![alt image](https://i.imgur.com/ngL4MTe.png)


### Rodzaje widoków

Django posiada dwa rodzaje pisania widoków:
* **function-based views** (FBVs) - widoki funkcyjne
* **class-based views** (CBVs) - widoki klasowe

Na początku Django posiadało tylko widoki funkcyjne, potem dodali również widoki klasowe, żeby ułatwić pisanie często powtarzającej się logiki w naszych widokach (dodawanie danych do bazy, dodawanie formularza, lista elementów z bazy itp.). Zarówno widoki funkcyjne jak i klasowe mają swoje zalety i wady, dlatego nie możemy mówić, że należy używać tylko jednej metody podczas pisania naszych widoków.

### Materiały
* [Class based views vs. function based views](https://medium.com/@ksarthak4ever/django-class-based-views-vs-function-based-view-e74b47b2e41b)